# Extract Annual Means for xqhuj from Raw Monthly Files

This notebook extracts annual means from raw monthly UM output files in `~/dump2hold/xqhuj/datam/`

In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import iris

sys.path.append(os.path.expanduser('~/scripts/utils_cmip7'))
from analysis import (
    find_matching_files,
    try_extract,
    compute_monthly_mean,
    merge_monthly_results,
    stash
)

## 1. Find Raw Monthly Files

In [ ]:
expt_name = 'xqhuj'

files = find_matching_files(
    expt_name=expt_name,
    model='a',
    up='pi',
    start_year=None,
    end_year=None,
    base_dir='~/dump2hold',
)

print(f"Found {len(files)} monthly files")
print(f"First file: {files[0] if files else 'None'}")
print(f"Last file: {files[-1] if files else 'None'}")

## 2. Extract Single Variable (GPP Example)

In [ ]:
# Extract GPP from all monthly files
monthly_results = []

for y, m, f in files:
    cubes = iris.load(f)
    cube = try_extract(cubes, 'gpp', stash_lookup_func=stash)
    
    if cube:
        mm = compute_monthly_mean(cube[0], 'GPP')
        monthly_results.append(mm)

print(f"Processed {len(monthly_results)} files for GPP")

## 3. Merge into Annual Means

In [ ]:
# Merge monthly results into annual means
gpp_annual = merge_monthly_results(monthly_results)

print(f"GPP annual means:")
print(f"  Years: {len(gpp_annual['years'])}")
print(f"  Range: {gpp_annual['years'][0]} - {gpp_annual['years'][-1]}")
print(f"  Units: {gpp_annual.get('units', 'N/A')}")

## 4. Plot GPP Time Series

In [ ]:
# Plot GPP (drop first year for spinup)
years = gpp_annual['years'][1:]
values = gpp_annual['data'][1:]

plt.figure(figsize=(10, 4))
plt.plot(years, values, linewidth=0.8)
plt.xlabel('Year')
plt.ylabel('GPP (PgC/year)')
plt.title(f'Gross Primary Production - {expt_name}')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Extract Multiple Variables

In [ ]:
# Variables to extract
variables = {
    'gpp': 'GPP',
    'npp': 'NPP',
    'rh': 'S resp',
    'cv': 'V carb',
    'cs': 'S carb',
}

annual_means = {}

for var_code, var_name in variables.items():
    print(f"\nProcessing {var_name}...")
    monthly_results = []
    
    for y, m, f in files:
        try:
            cubes = iris.load(f)
            cube = try_extract(cubes, var_code, stash_lookup_func=stash)
            
            if cube:
                mm = compute_monthly_mean(cube[0], var_name)
                monthly_results.append(mm)
        except Exception as e:
            continue
    
    if monthly_results:
        annual_means[var_code] = merge_monthly_results(monthly_results)
        print(f"  ✓ {var_name}: {len(annual_means[var_code]['years'])} years")
    else:
        print(f"  ❌ {var_name}: No data")

print(f"\nExtracted {len(annual_means)}/{len(variables)} variables")

## 6. Plot All Variables

In [ ]:
n_vars = len(annual_means)
fig, axes = plt.subplots(n_vars, 1, figsize=(10, 4*n_vars), sharex=True)

if n_vars == 1:
    axes = [axes]

for ax, (var_code, var_name) in zip(axes, variables.items()):
    if var_code not in annual_means:
        continue
    
    data = annual_means[var_code]
    years = data['years'][1:]  # Drop first year
    values = data['data'][1:]
    
    ax.plot(years, values, linewidth=0.8)
    ax.set_ylabel(var_name)
    ax.set_title(f"{var_name} - {expt_name}")
    ax.grid(True, alpha=0.3)

axes[-1].set_xlabel('Year')
plt.tight_layout()
plt.show()